# Decorators

There are various notebooks and modules in this project related to decorators, most starting with `dec`, e.g., `decorators.py`, `decgeneralize.ipynb`. See also `functions.py`, which touches on the meaning of the `@` syntax, since it is part of the syntax of function (and class) definitions, that is, of `def` (and `class`) statements.

This notebook contains general information, and also goes into some design details of code in `decorators.py`, including some details that are not inherently about the topic of decorators.

For decorators in this project that perform caching, see `caching.py`.

## A motivating example: manual wrapping

In [1]:
def square(n):
    return n**2

In [2]:
def wrapper(n):
    print(f'{square.__name__}({n})')
    return square(n)

In [3]:
wrapper(3)

square(3)


9

## Manual decorator application

Decorators are higher-order functions (or other callables) and one can call them explicitly, rather than using the `@` syntax.

In [4]:
from algoviz.decorators import peek_arg

In [5]:
psquare = peek_arg(square)

In [6]:
# List comprehension of the squares of 0 through 4 (inclusive).
# Use psquare in the list comprehension. This will have the side effect
# of printing information about the five calls to square.
[psquare(x) for x in range(5)]

square(0)
square(1)
square(2)
square(3)
square(4)


[0, 1, 4, 9, 16]

In [7]:
def fibonacci(n):
    """Compute the Fibonacci number F(n)."""
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [8]:
pfibonacci = peek_arg(fibonacci)

In [9]:
pfibonacci(4)

fibonacci(4)


3

In [10]:
square = peek_arg(square)

In [11]:
[square(x) for x in range(5)]

square(0)
square(1)
square(2)
square(3)
square(4)


[0, 1, 4, 9, 16]

In [12]:
fibonacci = peek_arg(fibonacci)

In [13]:
fibonacci(4)

fibonacci(4)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(0)
fibonacci(1)
fibonacci(2)
fibonacci(1)
fibonacci(0)


3

In [14]:
# Just for clarity. None of the del statements in this notebook should be necessary.
del square
del fibonacci

## Decorated function definitions

Here we use `@decorators.peek_arg` to show function calls, including recursive calls.

In [15]:
@peek_arg
def square(n):
    return n**2

In [16]:
square(7)

square(7)


49

In [17]:
@peek_arg
def fibonacci(n):
    """Compute the Fibonacci number F(n)."""
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [18]:
fibonacci(4)

fibonacci(4)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(0)
fibonacci(1)
fibonacci(2)
fibonacci(1)
fibonacci(0)


3

In [19]:
def use_locally(f):
    fibonacci = 'famous Italian mathematician'
    f(4)

use_locally(fibonacci)

fibonacci(4)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(0)
fibonacci(1)
fibonacci(2)
fibonacci(1)
fibonacci(0)


In [20]:
def define_and_use_locally():
    @peek_arg
    def sq(n):
        return n**2
    
    @peek_arg
    def fib(n):
        if n == 0:
            return 0
        if n == 1:
            return 1
        return fib(n - 1) + fib(n - 2)
    
    print([sq(x) for x in range(5)])
    print()
    print(fib(4))
    print()
    use_locally(fib)

define_and_use_locally()

sq(0)
sq(1)
sq(2)
sq(3)
sq(4)
[0, 1, 4, 9, 16]

fib(4)
fib(3)
fib(2)
fib(1)
fib(0)
fib(1)
fib(2)
fib(1)
fib(0)
3

fib(4)
fib(3)
fib(2)
fib(1)
fib(0)
fib(1)
fib(2)
fib(1)
fib(0)


## Non-wrapping decorators

The `@decorators.call` decorator does not wrap `func`, but instead calls it and then simply passes it along so it is bound to the name (variable) just as it would be if it had been defined undecorated.

In [21]:
from algoviz.decorators import call

In [22]:
@call
def hi():
    print('Hi, world!')

Hi, world!


In [23]:
hi()

Hi, world!


In [24]:
@call
def define_and_use_locally():
    @peek_arg
    def sq(n):
        return n**2
    
    @peek_arg
    def fib(n):
        if n == 0:
            return 0
        if n == 1:
            return 1
        return fib(n - 1) + fib(n - 2)
    
    print([sq(x) for x in range(5)])
    print()
    print(fib(4))
    print()
    use_locally(fib)

sq(0)
sq(1)
sq(2)
sq(3)
sq(4)
[0, 1, 4, 9, 16]

fib(4)
fib(3)
fib(2)
fib(1)
fib(0)
fib(1)
fib(2)
fib(1)
fib(0)
3

fib(4)
fib(3)
fib(2)
fib(1)
fib(0)
fib(1)
fib(2)
fib(1)
fib(0)


## Stacking decorators

We can apply any number of decorations to a function (or class) definition. The function (or class) is passed up through them bottom to top, much as how in `f(g(h(x)))`, the functions `f`, `g`, and `h` are called inside out (which, due to the prefix application syntax, is right to left: `h`, then `g`, then `f`).

So, a function with two decorations is created, passed to the lower decorator, then whatever that decorator returns is passed to the upper decorator, then whatever that returns is bound to the name (that the function would’ve been bound to immediately if its definition had not been decorated).

It works the same, but with more calls, still down-to-up, when there are more than two decorators and/or when the decorations are on a class definition rather than a function definition.

In [25]:
from algoviz.decorators import peek_return

In [26]:
@peek_return
@peek_arg
def fibonacci(n):
    """Compute the Fibonacci number F(n)."""
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [27]:
fibonacci(5)

fibonacci(5)
fibonacci(4)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(1)
fibonacci(1) -> 1
fibonacci(3) -> 2
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(4) -> 3
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(1)
fibonacci(1) -> 1
fibonacci(3) -> 2
fibonacci(5) -> 5


5

In [28]:
@peek_arg
@peek_return
def fibonacci(n):
    """Compute the Fibonacci number F(n)."""
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [29]:
fibonacci(5)

fibonacci(5)
fibonacci(4)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(1)
fibonacci(1) -> 1
fibonacci(3) -> 2
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(4) -> 3
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(1)
fibonacci(1) -> 1
fibonacci(3) -> 2
fibonacci(5) -> 5


5

#### Old *to-do*s and their resolutions:

Fix the problem with `wrapper` being the `__name__` (and other attributes):

- `@decorators.give_metadata_from` (exercise)
- `@functools.wraps` (to usually use)

Elucidate the connection to memoization:

- `@caching.memoize` (exercise):
- `@caching.memoize_by` (exercise + useful)
- `@functools.cache` (to usually use)

## More examples of the effect of the order of decorators

These examples use a wrapping decorator with a non-wrapping decorator.

In [30]:
from algoviz.decorators import thrice

In [31]:
@call
@thrice
def hi():
    print('Hi!')

Hi!
Hi!
Hi!


In [32]:
@thrice
@call
def hi():
    print('Hi!')

Hi!


In [33]:
hi()

Hi!
Hi!
Hi!


In [34]:
from algoviz.decorators import repeat

In [35]:
@call
@repeat(9)
def scotus():
    print('I am the law!')

I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!
I am the law!


## Metadata attributes on wrappers

In [36]:
def f():
    """A function that does nothing interesting."""
    pass

In [37]:
help(f)

Help on function f in module __main__:

f()
    A function that does nothing interesting.



In [38]:
f.__doc__

'A function that does nothing interesting.'

In [39]:
@peek_return
def g():
    """Another function that does nothing interesting."""
    pass

In [40]:
help(g)

Help on function g in module __main__:

g(arg)
    Another function that does nothing interesting.



In [41]:
g.__doc__

'Another function that does nothing interesting.'

In [42]:
dir(f)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

We don’t want to assign all attributes from the wrapped function the wrapper function. Let’s assign just these attributes:

- `__module__`
- `__name__`
- `__qualname__`
- `__doc__`
- `__annotations__`

In [43]:
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(arg)
    Compute the Fibonacci number F(n).



In [44]:
from algoviz import dechello

Hello, World!


In [45]:
dechello.greet('Bob')

greet('Bob')
Hello, Bob! Do you like decorators?


In [46]:
help(dechello.greet)

Help on function greet in module algoviz.dechello:

greet(name)
    Greet a user, printing information about the call.
    
    >>> greet('Bob')
    greet('Bob')
    Hello, Bob! Do you like decorators?



In [47]:
@thrice
def say_hi():
    """Say hi three times."""
    print('Hi.')

In [48]:
say_hi()

Hi.
Hi.
Hi.


In [49]:
help(say_hi)

Help on function say_hi in module __main__:

say_hi()
    Say hi three times.



In [50]:
f = lambda x: x**2

In [51]:
help(f)

Help on function <lambda> in module __main__:

<lambda> lambda x



In [52]:
f.__name__

'<lambda>'

In [53]:
f.__doc__

In [54]:
f.__qualname__

'<lambda>'

## Example of decoration order not affecting behavior

In [55]:
def prepend(prefix, text):
    return prefix + text

def append(text, suffix):
    return text + suffix

In [56]:
prepend('foo', append('bar', 'baz'))

'foobarbaz'

In [57]:
append(prepend('foo', 'bar'), 'baz')

'foobarbaz'

For `@peek_arg`, a decorated function such as `fibonacci`, and `@peek_return`, a closer (actually, almost exact) analogy is:

- Do the first thing. Then, do the second thing, and then the third thing.

- Do the first thing, and then the second thing. Then, do the third thing.

In [58]:
@peek_arg
@peek_return
def fibonacci_a(n):
    """Do the first thing. Then, do the second thing, and then the third thing."""
    return n if n < 2 else fibonacci_a(n - 2) + fibonacci_a(n - 1)

In [59]:
@peek_return
@peek_arg
def fibonacci_b(n):
    """Do the first thing, and then the second thing. Then, do the third thing."""
    return n if n < 2 else fibonacci_b(n - 2) + fibonacci_b(n - 1)

## `@peek` instead of using both `@peek_arg` and `@peek_return`

Since these decorators wrap, another frame is used on the call stack for each one. This is one of the reasons we may prefer to combine the functionality of separate but closely related wrapping decorators that we intend to use together comparably often as we intend to use separately. When we want `@peek_arg` or `@peek_return` only, we use just the one we want. When we want both effects, we can apply both (if it does not cause excessive stack depth in our use case), but we can and will typically just use `@peek`.

In [60]:
from algoviz.decorators import peek

In [61]:
@peek
def fibonacci(n):
    """Compute the Fibonacci number F(n)."""
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [62]:
fibonacci(5)

fibonacci(5)
fibonacci(4)
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(1)
fibonacci(1) -> 1
fibonacci(3) -> 2
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(4) -> 3
fibonacci(3)
fibonacci(2)
fibonacci(1)
fibonacci(1) -> 1
fibonacci(0)
fibonacci(0) -> 0
fibonacci(2) -> 1
fibonacci(1)
fibonacci(1) -> 1
fibonacci(3) -> 2
fibonacci(5) -> 5


5

In [63]:
from algoviz.fibonacci import fibonacci_cached_4

In [64]:
# fibonacci_cached_4(500)

### Examining the maximum depth of the call stack

In [65]:
import sys

In [66]:
sys.getrecursionlimit()

3000

It can be set with `sys.setrecursionlimit`, but it is important to be careful, because the C call stack may be, and, up to and including Python 3.10, in practice always is, used for a Python function call, so we can get a stack overflow (and a crash, or possibly even an exploitable stack-smashing vulnerability) if we excessively weaken this guardrail.

In Python 3.11, function calls within Python code (not calling into anything implemented in native code) are not achieved by a C function call in the interpreter. So in Python 3.11 (or at least in CPython 3.11), there are broad situations where it is safe to set the “recursion limit” extremely high, but one still needs to be careful.

## Instrumenting recursive binary search

In [67]:
def binary_search(values, x):
    @peek
    def help_binary(low, high):  # high is an inclusive endpoint.
        if low > high:
            return None
        halfway = (low + high) // 2
        if x > values[halfway]:
            return help_binary(halfway + 1, high)
        if x < values[halfway]:
            return help_binary(low, halfway - 1)
        return halfway  # values[halfway] should = x, possibly add assert.

    return help_binary(0, len(values) - 1)

In [68]:
binary_search([1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 16, 17, 19], 16)

help_binary(0, 13)
help_binary(7, 13)
help_binary(11, 13)
help_binary(11, 11)
help_binary(11, 11) -> 11
help_binary(11, 13) -> 11
help_binary(7, 13) -> 11
help_binary(0, 13) -> 11


11

In [69]:
binary_search(range(-3371, 10_000_000, 242), 4_776_001)

help_binary(0, 41336)
help_binary(0, 20667)
help_binary(10334, 20667)
help_binary(15501, 20667)
help_binary(18085, 20667)
help_binary(19377, 20667)
help_binary(19377, 20021)
help_binary(19700, 20021)
help_binary(19700, 19859)
help_binary(19700, 19778)
help_binary(19740, 19778)
help_binary(19740, 19758)
help_binary(19750, 19758)
help_binary(19750, 19753)
help_binary(19750, 19750)
help_binary(19750, 19749)
help_binary(19750, 19749) -> None
help_binary(19750, 19750) -> None
help_binary(19750, 19753) -> None
help_binary(19750, 19758) -> None
help_binary(19740, 19758) -> None
help_binary(19740, 19778) -> None
help_binary(19700, 19778) -> None
help_binary(19700, 19859) -> None
help_binary(19700, 20021) -> None
help_binary(19377, 20021) -> None
help_binary(19377, 20667) -> None
help_binary(18085, 20667) -> None
help_binary(15501, 20667) -> None
help_binary(10334, 20667) -> None
help_binary(0, 20667) -> None
help_binary(0, 41336) -> None


## Decorators do not *re*bind names

People often wrongly claim that the name of a decorated function is bound twice—once to the function as initially defined, then a second time to what the decorator returns when called with that original function. That is, they say the decorated function name is assigned to twice: *re*bound.

That is, when considering code like this:

In [70]:
@thrice
def parrot():
    print('a parrot')

In [71]:
parrot()

a parrot
a parrot
a parrot


In [72]:
del parrot

...People often claim that it is *equivalent to*, or even *syntactic sugar for*, this code:

In [73]:
def parrot():
    print('a parrot')

parrot = thrice(parrot)

In [74]:
parrot()

a parrot
a parrot
a parrot


In [75]:
del parrot

Showing both versions of the code above is a good way to motivate a reader/student’s thinking when introducing decorators (and an approach much like that is taken at the top of this notebook). But decorated function definitions, like undecorated function definitions, bind a name exactly once.

The only object assigned to the function name is whatever the decorator returned—or, with multiple decorators, whatever the last-called (i.e., topmost) decorator returned. This is to say that, in a `def` statement, a function is not assigned to a name until all decorators have been called; so when decorators are used, the original function is *never* assigned to a name, unless it is itself returned by the (topmost) decorator.

The official documentation *mostly* manages to avoid the mistake. In particular, the language reference, which is the most authoritative source about the meaning of Python language constructs, is clear and correct:

> A function definition may be wrapped by one or more [decorator](https://docs.python.org/3/glossary.html#term-decorator) expressions. Decorator expressions are evaluated when the function is defined, in the scope that contains the function definition. The result must be a callable, which is invoked with the function object as the only argument. The returned value is bound to the function name **instead of** the function object. Multiple decorators are applied in nested fashion. For example, the following code
> 
> ```python
> @f1(arg)
> @f2
> def func(): pass
> ```
>
> <br>is roughly equivalent to
>
> ```python
> def func(): pass
> func = f1(arg)(f2(func))
> ```
> 
> <br>except that **the original function is not temporarily bound to the name func**.

*— [The Python Language Reference](https://docs.python.org/3/reference/index.html), section [8.7. Function definitions](https://docs.python.org/3/reference/compound_stmts.html#function-definitions), **bold** added*

[PEP 318 – Decorators for Functions and Methods](https://peps.python.org/pep-0318/), which covers this in the [Current Syntax](https://peps.python.org/pep-0318/#current-syntax) section, is likewise accurate and unambiguous.

Yet the false belief that decorators rebind names is remarkably persistent, even among many otherwise highly proficient Python programmers, and even among successful consultants and respected authors. As of this writing, at least one *official* source—the [glossary entry](https://docs.python.org/3/glossary.html#term-decorator), which the language reference unfortunately links to—makes the false “syntactic sugar” claim. So do most unofficial Python resources.

It may be that the misconception survives because people think there is no observable difference between binding the name once or twice, or think that explanations in terms of rebinding are *conceptually* sufficient even though such rebinding is guaranteed never to occur. Unfortunately, those are misconceptions as well: it is possible to write simple code that demonstrates that the function name is not assigned before the decorator is called, and one of the most common uses of decorators relies on this behavior.

### Exercise showing that decorators do not rebind names

As an exercise, please write a simple example below whose observable behavior reveals that a decorated `def` statement binds the function name just once, after the decorator is called, rather than twice, both before and after the decorator is called. The goal here is simplicity and clarity—it is okay if the example is contrived.

As for the common (non-contrived) use of decorators that relies on this behavior, we'll soon cover that in `classes2.ipynb`—though actually you've both seen and used it before, it being so common—which is one of the reasons I wanted to review decorator semantics and pose this exercise at this time.

In [76]:
import functools

In [77]:
del f

In [78]:
def raises_exception(func): 
    raise ValueError

In [79]:
@raises_exception
def f():
    pass

ValueError: 

In [80]:
print(f)

NameError: name 'f' is not defined

## Scratchwork for `@decorators.count_calls_in_attribute`

In [81]:
import functools

In [82]:
def count_calls(func): 
    @functools.wraps(func)
    def wrapper(*pargs, **kwargs): 
        if hasattr(wrapper, 'count'): 
            count = getattr(wrapper, 'count')
        else: 
            count = 0
        count += 1
        setattr(wrapper, 'count', count)
        return func(*pargs, **kwargs)
    return wrapper

In [83]:
def c(): return 1

In [84]:
@count_calls
def d(): return 1

In [85]:
d()

1

In [86]:
d.count

1

In [87]:
d()

1

In [88]:
d.count

2

In [89]:
d()

1

In [90]:
d.count

3

In [91]:
def count_calls_factory(*, name='default'): 
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*pargs, **kwargs):
            if hasattr(wrapper, name):
                count = getattr(wrapper, name)
            else:
                count = 0

            count += 1
            setattr(wrapper, name, count)

            return func(*pargs, **kwargs)

        return wrapper
    return decorator

In [92]:
@count_calls_factory()
def e(): return 1

In [93]:
e()

1

In [94]:
e.default

1

In [95]:
e()

1

In [96]:
e.default

2

## `@decorators.linear_combinable` and inheritance

`@linear_combinable` does not document what type it returns, and the implementation is permitted to vary considerably, through in practice there are only a small number of ways to implement it correctly. Rather than assuming anything about this type, I use `type` to obtain it, then attempt to inherit from it. If defining the derived class were to fail, that would be acceptable (so long as it failed in a way that made sense, such as with an error about the type not being possible to inherit from). In practice, however, it will be possible to inherit from it.

Since inheritance is a relationship of very close coupling, and the exercise does not impose restrictions on the type `linear_combinable` returns, arguably it is simply a bug to attempt to inherit from the type of the instance returned, and therefore not the responsibility of the implementer of `@linear_combinable` to make that work in a reasonable way. However, there are some design choices where, with a small amount of additional design effort and either no additional code or a small amount of additional code, inheritance can be made to work fine.

### Base and derived class instances for testing

In [97]:
from algoviz.decorators import identity_function, linear_combinable

In [98]:
base_instance = linear_combinable(identity_function)

In [99]:
LC = type(base_instance)

In [100]:
class MyLC(LC):
    pass

In [101]:
derived_instance = MyLC(identity_function)

### Equality comparison works automatically.

In [102]:
# This is what we want, as MyLC has not further customized equality comparison.
base_instance == derived_instance  # Automatically works for any reasonable way of checking types.

True

In [103]:
# This is likewise correct.
derived_instance == base_instance  # Automatically works for any reasonable way of checking types.

True

### But addition and subtraction do not always work automatically.

In [104]:
base_instance + derived_instance  # Automatically works for any reasonable way of checking types.

linear_combinable(<function linear_combinable.__add__.<locals>.<lambda> at 0x0000020FBA86F420>)

In [105]:
derived_instance + base_instance  # Works if we implement __radd__ or don't check against type(self).

linear_combinable(<function linear_combinable.__radd__.<locals>.<lambda> at 0x0000020FBA86F920>)

In [106]:
base_instance - derived_instance  # Automatically works for any reasonable way of checking types.

linear_combinable(<function linear_combinable.__sub__.<locals>.<lambda> at 0x0000020FBA86FE20>)

In [107]:
derived_instance - base_instance  # Works if we implement __rsub__ or don't check against type(self).

linear_combinable(<function linear_combinable.__rsub__.<locals>.<lambda> at 0x0000020FBA3244A0>)

### Operations between instances of unrelated derived classes

Note that if we have two separately derived classes, further design decisions are needed about whether those comparisons should work and what they should do. This relates to whether we check against the base class `linear_combinable` or against `type(self)`.

In [108]:
class MyOtherLC(LC):
    pass

In [109]:
other_derived_instance = MyOtherLC(identity_function)

In [110]:
derived_instance == other_derived_instance

False

In [111]:
other_derived_instance == derived_instance

False

In [112]:
derived_instance + other_derived_instance

TypeError: unsupported operand type(s) for +: 'MyLC' and 'MyOtherLC'

In [113]:
other_derived_instance + derived_instance

TypeError: unsupported operand type(s) for +: 'MyOtherLC' and 'MyLC'

#### Unfortunately, `==` is intransitive in the general case:

In [114]:
derived_instance == other_derived_instance

False

In [115]:
derived_instance == base_instance == other_derived_instance

True

But it may still be an acceptable price to pay for avoiding complicated code or complicated behavior, especially if we are not designing for inheritance.